In [ ]:
# 모델 매개변수 최적화하기

# 이제 모델과 데이터가 준비되었으니, 데이터에 매개변수를 최적화하여 모델을 학습하고, 검증하고, 테스트할 차례입니다.
# 모델을 학습하는 과정은 반복적인 과정을 거칩니다; 각 반복 단계에서 모델은 출력을 추측하고, 추측과 정답 사이의 오류(손실(loss))를 계산하고,
# (이전 장에서 본 것처럼) 매개변수에 대한 오류의 도함수(derivative)를 수집한 뒤,
# 경사하강법을 사용하여 이 파라미터들을 최적화(optimize)합니다.

import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

100%|██████████| 26421880/26421880 [00:03<00:00, 6914796.95it/s] 


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 121414.19it/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:02<00:00, 2158721.24it/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 14149591.74it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [ ]:
# 하이퍼파라미터(Hyperparameter)

# 하이퍼파라미터(Hyperparameter)는 모델 최적화 과정을 제어할 수 있는 조절 가능한 매개변수입니다.
# 서로 다른 하이퍼파라미터 값은 모델 학습과 수렴율(convergence rate)에 영향을 미칠 수 있습니다.

# 학습 시에는 다음과 같은 하이퍼파라미터를 정의합니다:
# 에폭(epoch) 수 - 데이터셋을 반복하는 횟수

# 배치 크기(batch size) - 매개변수가 갱신되기 전 신경망을 통해 전파된 데이터 샘플의 수

# 학습률(learning rate) - 각 배치/에폭에서 모델의 매개변수를 조절하는 비율. 값이 작을수록 학습 속도가 느려지고, 값이 크면 학습 중 예측할 수 없는 동작이 발생할 수 있습니다.

learning_rate = 1e-3
batch_size=64
epochs=5

In [ ]:
# 최적화 단계(Optimization Loop)

# 하이퍼파라미터를 설정한 뒤에는 최적화 단계를 통해 모델을 학습하고 최적화할 수 있습니다. 최적화 단계의 각 반복(iteration)을 에폭이라고 부릅니다.

# 하나의 에폭은 다음 두 부분으로 구성됩니다:
# 학습 단계(train loop) - 학습용 데이터셋을 반복(iterate)하고 최적의 매개변수로 수렴합니다.

# 검증/테스트 단계(validation/test loop) - 모델 성능이 개선되고 있는지를 확인하기 위해 테스트 데이터셋을 반복(iterate)합니다.

In [ ]:
# 손실 함수(loss function)

# 학습용 데이터를 제공하면, 학습되지 않은 신경망은 정답을 제공하지 않을 확률이 높습니다.
# 손실 함수(loss function)는 획득한 결과와 실제 값 사이의 틀린 정도(degree of dissimilarity)를 측정하며, 학습 중에 이 값을 최소화하려고 합니다.
#  주어진 데이터 샘플을 입력으로 계산한 예측과 정답(label)을 비교하여 손실(loss)을 계산합니다.

# 손실 함수 초기화

loss_fn = nn.CrossEntropyLoss()

In [ ]:
# 옵티마이저(Optimizer)

# 최적화는 각 학습 단계에서 모델의 오류를 줄이기 위해 모델 매개변수를 조정하는 과정입니다.
# 최적화 알고리즘은 이 과정이 수행되는 방식(여기에서는 확률적 경사하강법(SGD; Stochastic Gradient Descent))을 정의합니다

# 학습하려는 모델의 매개변수와 학습률(learning rate) 하이퍼파라미터를 등록하여 옵티마이저를 초기화합니다.

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)



In [ ]:
# 학습 단계(loop)에서 최적화는 세단계로 이뤄집니다:
# optimizer.zero_grad()를 호출하여 모델 매개변수의 변화도를 재설정합니다. 기본적으로 변화도는 더해지기(add up) 때문에 중복 계산을 막기 위해 반복할 때마다 명시적으로 0으로 설정합니다.

# loss.backwards()를 호출하여 예측 손실(prediction loss)을 역전파합니다. PyTorch는 각 매개변수에 대한 손실의 변화도를 저장합니다.

# 변화도를 계산한 뒤에는 optimizer.step()을 호출하여 역전파 단계에서 수집된 변화도로 매개변수를 조정합니다.

In [ ]:
# 전체 구현

# 최적화 코드를 반복하여 수행하는 train_loop와 테스트 데이터로 모델의 성능을 측정하는 test_loop를 정의하였습니다.

def train_loop(dataloader,model,loss_fn,optimizer):
  size=len(dataloader.dataset) #데이터셋의 전체 샘플 수입니다.
  for batch, (X,y) in enumerate(dataloader):#반복문은 데이터로더의 배치(batch)마다 실행되며, 각 배치는 입력 특성 X와 해당하는 레이블 y로 구성됩니다.

    pred=model(X) #모델에 입력 X를 전달한 결과인 예측값이 저장됩니다.
    loss = loss_fn(pred,y) # 예측값 pred와 실제 레이블 y 사이의 손실을 계산한 값입니다.

    optimizer.zero_grad() #매 반복마다 기울기를 0으로 초기화합니다. 이는 각 배치마다 새로운 기울기를 계산하기 위해 필요합니다.
    loss.backward() # 손실에 대한 기울기를 계산합니다. 이를 통해 역전파(Backpropagation) 알고리즘을 수행하여 모델의 각 매개변수에 대한 기울기를 구합니다.
    optimizer.step() #계산된 기울기를 사용하여 모델의 매개변수를 업데이트합니다. 최적화기(optimizer)에 따라 학습 알고리즘이 수행됩니다.

    if batch % 100 == 0:
      loss, current = loss.item(), (batch+1) * len(X) #매 100번째 배치마다 현재 손실(loss)과 현재까지 처리한 총 샘플 수(current)를 출력합니다. 이를 통해 학습 과정을 모니터링할 수 있습니다.
      print(f'loss: {loss:>7f} [{current:>5d}/{size:>5d}]')


def test_loop(dataloader,model,loss_fn):
  size = len(dataloader.dataset) #데이터셋의 전체 샘플 수입니다.
  num_batches=len(dataloader) #데이터로더의 총 배치 수입니다.
  test_loss, correct = 0,0


# with 블록 내에서는 기울기 계산을 비활성화합니다. 이는 테스트 단계에서는 모델을 평가만 하기 때문에 기울기가 필요하지 않기 때문입니다.
# 반복문을 통해 테스트 데이터로 모델의 예측을 수행하고, 손실과 정확도를 누적합니다.
  with torch.no_grad():
    for X,y in dataloader:
      pred=model(X)
      test_loss += loss_fn(pred, y).item()
      correct += (pred.argmax(1)==y).type(torch.float).sum().item()

  test_loss/=num_batches #평균 손실도 계산
  correct/=size #평균 정확도 계산

  print(f"Test Error: \n Accuracy : {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
#실제 동작부

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
  print(f"Epoch {t+1}\n------------------------")
  train_loop(train_dataloader, model, loss_fn, optimizer)
  test_loop(test_dataloader, model, loss_fn)

print("Done!")

Epoch 1
------------------------
loss: 2.299146 [   64/60000]
loss: 2.293664 [ 6464/60000]
loss: 2.263869 [12864/60000]
loss: 2.248427 [19264/60000]
loss: 2.266222 [25664/60000]
loss: 2.211975 [32064/60000]
loss: 2.222844 [38464/60000]
loss: 2.183838 [44864/60000]
loss: 2.180999 [51264/60000]
loss: 2.145195 [57664/60000]
Test Error: 
 Accuracy : 40.8%, Avg loss: 2.148309 

Epoch 2
------------------------
loss: 2.162261 [   64/60000]
loss: 2.156448 [ 6464/60000]
loss: 2.091113 [12864/60000]
loss: 2.093278 [19264/60000]
loss: 2.074524 [25664/60000]
loss: 1.998371 [32064/60000]
loss: 2.018163 [38464/60000]
loss: 1.942746 [44864/60000]
loss: 1.944183 [51264/60000]
loss: 1.858578 [57664/60000]
Test Error: 
 Accuracy : 57.5%, Avg loss: 1.873281 

Epoch 3
------------------------
loss: 1.912019 [   64/60000]
loss: 1.885334 [ 6464/60000]
loss: 1.760047 [12864/60000]
loss: 1.780693 [19264/60000]
loss: 1.699296 [25664/60000]
loss: 1.644693 [32064/60000]
loss: 1.650373 [38464/60000]
loss: 1.5628